# Create Kubernetes cluster and deploy HF model

## Reference article
https://getindata.com/blog/deploy-open-source-llm-private-cluster-hugging-face-gke-autopilot/

## Prerequisites
git-lfs
100 GB of free space on disk

In [ ]:
## Setting required env variables

%env S3_BUCKET_NAME=k8s-model-zephyr
%env REGION=eu-central-1
%env HF_MODEL_PATH=HuggingFaceH4/zephyr-7b-alpha
%env HF_MODEL_NAME=zephyr-7b-alpha
%env LOCAL_DIRECTORY=/data-tst/home/voa/projects/k8s-model
%env AWS_PROFILE voatsap-cluster-dev

In [ ]:
# clone model to local folder and upload to s3 bucket
!cd $LOCAL_DIRECTORY
!git lfs clone --depth=1 https://huggingface.co/$HF_MODEL_PATH
!aws s3 mb s3://$S3_BUCKET_NAME --region $REGION

In [ ]:
# copy HF model to s3 bucket
!aws s3 sync $LOCAL_DIRECTORY/$HF_MODEL_NAME s3://$S3_BUCKET_NAME/llm/deployment/$HF_MODEL_NAME --exclude "*.git/*"

In [ ]:
# bootstrap cluster
!cd cluster.dev
!cdev apply

In [ ]:
# install nvidia drivers
!kubectl apply -f https://raw.githubusercontent.com/NVIDIA/k8s-device-plugin/main/nvidia-device-plugin.yml

In [ ]:
# deploy model files
!kubectl apply -f ../kubernetes/model/deployment.yaml

# port forward model service
!kubectl port-forward svc/zephyr-7b-alpha-service  8081:8080

!curl 127.0.0.1:8081/generate \
    -X POST \
    -d '{"inputs":"Continue funny story: John decide to stick finger into outlet","parameters":{"max_new_tokens":2000}}' \
    -H 'Content-Type: application/json'